<a href="https://colab.research.google.com/github/mentorbruh/digit-recognizer/blob/main/transfer_learning_try_out_mnist_fashion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import keras
from keras import layers
from keras.models import Model, Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D
from keras.utils import to_categorical
from keras.datasets import fashion_mnist


from keras.applications.vgg16 import VGG16
%matplotlib inline

In [2]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

In [3]:
X_train.shape

(60000, 28, 28)

# Image set up:
- 1. Convert images into 3 channels.
- 2. Reshape the image as per the tensor format.
- 3. Reshape the image into 48 * 48. This is a requiremet of VGG16.
- 4. Normalize the image data and change data type to float.

## Step 1:

In [4]:
X_train = np.dstack([X_train] * 3)
X_test = np.dstack([X_test] * 3)

## Step 2:

In [5]:
X_train = X_train.reshape(-1, 28,28,3)
X_test= X_test.reshape (-1,28,28,3)
X_train.shape, X_test.shape

((60000, 28, 28, 3), (10000, 28, 28, 3))

## Step 3:

In [6]:
from keras.preprocessing.image import img_to_array, array_to_img

X_train = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in X_train])
X_test = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in X_test])
#train_x = preprocess_input(x)
X_train.shape, X_test.shape

((60000, 48, 48, 3), (10000, 48, 48, 3))

## Step 4:

In [7]:
# Normalise the data and change data type
X_train = X_train / 255.
X_test = X_test/ 255.
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train.shape, X_test.shape

((60000, 48, 48, 3), (10000, 48, 48, 3))

In [8]:
# Converting Labels to one hot encoded format
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# VGG16

In [10]:
import keras
from keras import models
from keras import layers
from keras import optimizers
from keras.applications.vgg16 import VGG16
from keras.layers import Activation, Dense

In [11]:
# Define the parameters for the VGG16 model
IMG_WIDTH = 48
IMG_HEIGHT = 48
IMG_DEPTH = 3
BATCH_SIZE = 16

In [23]:
#This is the setup for the VGG16 network:

model_vgg16= VGG16( 
                  input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH),
                   weights = 'imagenet',
                   include_top = False
                 )

model_vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0     

In [24]:
for layer in model_vgg16.layers[:-4]:
    layer.trainable = False

The above code stops the model to trained again. 
we are doing this because the vgg16 is already trained model on lots of images.

In [25]:
model= Sequential()

model.add(model_vgg16)

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(128, activation = 'relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='sigmoid'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_7 (Dense)              (None, 128)               65664     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                1290      
Total params: 15,044,298
Trainable params: 7,409,034
Non-trainable params: 7,635,264
___________________________________

In [26]:
model.compile(
    loss='categorical_crossentropy',
    optimizer= 'adam',
    metrics=['acc'])

In [27]:
model.fit(X_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/10
469/469 [==============================] - 21s 45ms/step - loss: 0.7293 - acc: 0.7247 - val_loss: 0.4182 - val_acc: 0.8447
Epoch 2/10
469/469 [==============================] - 21s 45ms/step - loss: 0.4090 - acc: 0.8598 - val_loss: 0.4119 - val_acc: 0.8490
Epoch 3/10
469/469 [==============================] - 21s 45ms/step - loss: 0.3611 - acc: 0.8740 - val_loss: 0.3729 - val_acc: 0.8751
Epoch 4/10
469/469 [==============================] - 21s 44ms/step - loss: 0.3299 - acc: 0.8860 - val_loss: 0.3461 - val_acc: 0.8797
Epoch 5/10
469/469 [==============================] - 21s 45ms/step - loss: 0.3096 - acc: 0.8910 - val_loss: 0.4045 - val_acc: 0.8692
Epoch 6/10
469/469 [==============================] - 21s 45ms/step - loss: 0.2887 - acc: 0.8978 - val_loss: 0.3377 - val_acc: 0.8833
Epoch 7/10
469/469 [==============================] - 21s 45ms/step - loss: 0.2735 - acc: 0.9030 - val_loss: 0.3461 - val_acc: 0.8852
Epoch 8/10
469/469 [==============================] - 21s 45ms

Achieved 89.22% validation accuracy using vgg16 transfer learning model.

# ResNet50:

In [28]:
from keras.applications.resnet50 import ResNet50

In [29]:
model_resnet50= ResNet50( 
                  input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH),
                   weights = 'imagenet',
                   include_top = False
                 )

model_resnet50.summary()

94773248/94765736 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 48, 48, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 54, 54, 3)    0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 24, 24, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 24, 24, 64)   256         conv1_conv[0][0]                 
__________________________

In [30]:
for layer in model_resnet50.layers[:-4]:
    layer.trainable = False

In [33]:
from keras import layers
model_1= Sequential()

model_1.add(model_resnet50)
model_1.add(keras.layers.GlobalAveragePooling2D())
model_1.add(layers.Flatten())
model_1.add(layers.Dense(512, activation='relu'))
model_1.add(layers.Dropout(0.5))
model_1.add(layers.Dense(10, activation='sigmoid'))

model_1.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2, 2, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 512)               1049088   
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 10)                5130      
Total params: 24,641,930
Trainable params: 2,108,938
Non-trainable params: 22,532,992
__________________________________

In [34]:
model_1.compile(
    loss='categorical_crossentropy',
    optimizer= 'adam',
    metrics=['accuracy'])

In [35]:
model_1.fit(X_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/10
469/469 [==============================] - 17s 36ms/step - loss: 0.8177 - accuracy: 0.6997 - val_loss: 1.1707 - val_accuracy: 0.6407
Epoch 2/10
469/469 [==============================] - 16s 34ms/step - loss: 0.6205 - accuracy: 0.7699 - val_loss: 0.6567 - val_accuracy: 0.7562
Epoch 3/10
469/469 [==============================] - 16s 34ms/step - loss: 0.5710 - accuracy: 0.7896 - val_loss: 0.8499 - val_accuracy: 0.7011
Epoch 4/10
469/469 [==============================] - 16s 34ms/step - loss: 0.5403 - accuracy: 0.8004 - val_loss: 0.5820 - val_accuracy: 0.7792
Epoch 5/10
469/469 [==============================] - 16s 33ms/step - loss: 0.5179 - accuracy: 0.8087 - val_loss: 0.5802 - val_accuracy: 0.7847
Epoch 6/10
469/469 [==============================] - 16s 33ms/step - loss: 0.5015 - accuracy: 0.8152 - val_loss: 0.6383 - val_accuracy: 0.7682
Epoch 7/10
469/469 [==============================] - 16s 33ms/step - loss: 0.4843 - accuracy: 0.8198 - val_loss: 0.5425 - val_accuracy:

so using transfer learning with last 4 layers retrained only creates a performance of around 81-82% validation accuracy. On 30 epochs the model is a bit overtrained.

In [ ]:
for layer in model_resnet50.layers[:-1]:
    layer.trainable = False

In [ ]:
model_1= Sequential()

model_1.add(model_resnet50)
model_1.add(keras.layers.GlobalAveragePooling2D())
model_1.add(layers.Flatten())
model_1.add(layers.Dense(512, activation='relu'))
model_1.add(layers.Dropout(0.3))
model_1.add(layers.Dense(128, activation= 'relu'))
model_1.add(layers.Dropout(0.3))
model_1.add(layers.Dense(10, activation='softmax'))

model_1.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2, 2, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_5 ( (None, 2048)              0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 2048)              0         
_________________________________________________________________
dense_30 (Dense)             (None, 512)               1049088   
_________________________________________________________________
dropout_11 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 128)               65664     
_________________________________________________________________
dropout_12 (Dropout)         (None, 128)             

In [ ]:
model_1.compile(
    loss='categorical_crossentropy',
    optimizer= 'rmsprop',
    metrics=['accuracy'])

In [ ]:
model_1.fit(X_train, y_train,
          batch_size=128,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
469/469 [==============================] - 17s 36ms/step - loss: 0.5082 - accuracy: 0.8165 - val_loss: 0.5169 - val_accuracy: 0.8180
Epoch 2/5
469/469 [==============================] - 16s 35ms/step - loss: 0.3872 - accuracy: 0.8560 - val_loss: 0.4512 - val_accuracy: 0.8416
Epoch 3/5
469/469 [==============================] - 16s 34ms/step - loss: 0.3631 - accuracy: 0.8662 - val_loss: 0.4741 - val_accuracy: 0.8345
Epoch 4/5
469/469 [==============================] - 16s 34ms/step - loss: 0.3490 - accuracy: 0.8711 - val_loss: 0.4678 - val_accuracy: 0.8449
Epoch 5/5
469/469 [==============================] - 16s 34ms/step - loss: 0.3414 - accuracy: 0.8744 - val_loss: 0.4954 - val_accuracy: 0.8428


In adding one more dense layer to the end, and not changing the whole resnet model; we get 84% accuracy.
Let's check with simple resnet model once.

In [ ]:
for layer in model_resnet50.layers:
    layer.trainable = True

In [ ]:
model_1= Sequential()

model_1.add(model_resnet50)
model_1.add(layers.Flatten())
model_1.add(layers.Dense(10, activation='softmax'))

model_1.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2, 2, 2048)        23587712  
_________________________________________________________________
flatten_19 (Flatten)         (None, 8192)              0         
_________________________________________________________________
dense_39 (Dense)             (None, 10)                81930     
Total params: 23,669,642
Trainable params: 23,616,522
Non-trainable params: 53,120
_________________________________________________________________


In [ ]:
model_1.compile(
    loss='categorical_crossentropy',
    optimizer= 'rmsprop',
    metrics=['accuracy'])

In [ ]:
model_1.fit(X_train, y_train,
          batch_size=128,
          epochs=2,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/2
469/469 [==============================] - 51s 109ms/step - loss: 2.3027 - accuracy: 0.0979 - val_loss: nan - val_accuracy: 0.1000
Epoch 2/2
469/469 [==============================] - 51s 108ms/step - loss: 2.3027 - accuracy: 0.0966 - val_loss: nan - val_accuracy: 0.1000


check if vgg16 is even trainable. And no it is not. it doesn't achieve even slightest results because of vanishing gradient problem.